In [1]:
from rdflib import Graph, Namespace, URIRef
from typing import Optional
from pydantic import BaseModel
from api.src.function.loc.getType import GetType
from pyfuseki import FusekiUpdate

In [10]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

r = au_update.run_sparql('DROP DEFAULT')
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [111]:
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
class Agents(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    fullerName: Optional[Element]
    # note: Optional[str]
    # hasReciprocalAuthority: Optional[list[Uri]]
    # hasReciprocalExternalAuthority: Optional[list[Uri]]
    # hasBroaderAuthority: Optional[list[Uri]]
    # hasBroaderExternalAuthority: Optional[list[Uri]]
    # hasNarrowerAuthority: Optional[list[Uri]]
    # hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    # subjectOf: Optional[list[Uri]]
    # isMemberOfMADSCollection: str
    

In [18]:
from rdflib.collection import Collection
from rdflib import RDF, Literal

In [3]:
madsrdf = Namespace("http://www.loc.gov/mads/rdf/v1#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

In [2]:
authority = 'http://id.loc.gov/authorities/names/n80002329'
graph = Graph()
graph.parse(f'{authority}.rdf')

<Graph identifier=N4323f6a40bc04bdba55bcf4a379b6fe5 (<class 'rdflib.graph.Graph'>)>

In [ ]:
q = """PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?typeVariant ?elementList ?elementValue ?typeElement 
WHERE  {         	
<http://id.loc.gov/authorities/names/n80002329> madsrdf:hasVariant ?variant .
                ?variant rdf:type ?typeVariant .
                ?variant madsrdf:elementList ?elementList .
                ?elementList rdf:rest* ?node .
                  ?node rdf:first ?e .
                  ?e madsrdf:elementValue ?elementValue .
                ?e rdf:type ?typeElement .
                FILTER ( ?typeVariant != madsrdf:Variant )
                }"""
result = graph.query(q)     
   

In [38]:
result.bindings[0]

{rdflib.term.Variable('elementList'): rdflib.term.BNode('Nea87f9da482544feadcf8a50204d9486'), rdflib.term.Variable('typeVariant'): rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#PersonalName'), rdflib.term.Variable('elementValue'): rdflib.term.Literal('Assis, Joaquim Maria Machado de,'), rdflib.term.Variable('typeElement'): rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#FullNameElement')}

In [39]:
d = {'Nea87f9da482544feadcf8a50204d9486': ['x']}
for i in result.bindings:
    if i.get('elementList') in [d.keys()]:
        print(i.get('elementList')) 

In [48]:
def getElementList(graph, authority, obj):
    
  qElementList = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:elementList ?elementList .
    ?elementList rdf:rest* ?rest .
    ?rest rdf:first ?first .
    ?first rdf:type ?type  .
    ?first madsrdf:elementValue ?value
      }}"""
  r = graph.query(qElementList)
  bindings = r.bindings
  elementList = list()

  for i in bindings:
    element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('value').toPython(),
            "lang": i.get('lang')
          }
        }
    elementList.append(element)
  obj['elementList'] = elementList

  return obj

def getFullerName(graph, authority, obj):
    
  qFullerName = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:fullerName ?fullerName .
      ?fullerName rdf:type ?type .
      ?fullerName rdfs:label ?value
      }}"""
  r = graph.query(qFullerName)
  bindings = r.bindings
  if len(bindings) > 0:
    result = bindings[0]
    element = {
              "type": result.get('type').split("#")[1],
            "elementValue": {
              "value": result.get('value').toPython(),
              "lang": result.get('lang')
            }
          }
    obj['fullerName'] = element
    
  return obj

def GetUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadata} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value').toPython()
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
            external.append(uri)
        obj[metadata] = external
    return obj

def GetVariant(authority, graph, obj):

    qVariant = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
                <{authority}> madsrdf:hasVariant ?variant .
                ?variant rdf:type ?typeVariant .
                ?variant madsrdf:elementList ?elementList .
                ?elementList rdf:rest* ?node .
                  ?node rdf:first ?e .
                  ?e madsrdf:elementValue ?elementValue .
                ?e rdf:type ?typeElement .
                FILTER ( ?typeVariant != madsrdf:Variant )
                }}"""
    r = graph.query(qVariant)
    if len(r.bindings) > 0:

        variants = list()
        for i in r.bindings:
            value = i.get('elementValue').value
            variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': value.capitalize(),
                  'lang': 'pt'
              }
          }]
      }
            variants.append(variant)
        obj['hasVariant'] = variants
    return obj


In [107]:
madsrdf = Namespace("http://www.loc.gov/mads/rdf/v1#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

In [ ]:
type: str
elementList: Optional[list[Element]]

In [200]:
variants = graph.triples((URIRef(authority), madsrdf.hasVariant, None))
for _, _, o in variants:
    variant = graph.triples((o, None, None))
    for s, p, o in variant:
        if p == madsrdf.variantLabel:
            pass
            # print("VL: ",o)
        if p == madsrdf.elementList:
            elementList = graph.triples((o, None, None))
            for s, p, o in elementList:
                element = graph.triples((o, None, None))
                for s, p, o in element:
                    if p == rdf.type:
                        pass
                        # print(o)
                    elif p == madsrdf.elementValue:
                        pass
                        # print("E: ", o)
                    elif p == rdf.first:
                        first = graph.triples((o, None, None))
                        for s, p, o in first:
                            pass
                            # print(p, o)
                    else:
                        rest = graph.triples((o, None, None))
                        for s, p, o in rest:
                            print(p, o)
                    

                    
        

        


http://www.w3.org/1999/02/22-rdf-syntax-ns#first N3a5f2250a73e43dd9982275939d23898
http://www.w3.org/1999/02/22-rdf-syntax-ns#rest http://www.w3.org/1999/02/22-rdf-syntax-ns#nil


In [169]:
def getElement(graph, Node):
    e = {}
    triples = graph.triples((Node, None, None))
    for s, p, o in triples:
        if p == rdf.type:
                    e['type'] = o.split('#')[-1]
        elif p == madsrdf.elementValue:
                    e['elementValue'] = o.n3()
        else:
                print("P: ", p)
    return e





In [ ]:
elementList = graph.triples((variant, None, None))


In [106]:
elementList.send()

TypeError: throw expected at least 1 argument, got 0

In [89]:
variant = graph.triples((o, None, None))
for i in variant:
    print(i)

(rdflib.term.BNode('Nc947f08ba2154df8b104a95b228a757a'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#PersonalName'))
(rdflib.term.BNode('Nc947f08ba2154df8b104a95b228a757a'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#Variant'))
(rdflib.term.BNode('Nc947f08ba2154df8b104a95b228a757a'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#variantLabel'), rdflib.term.Literal('Semana Dr., 1839-1908'))
(rdflib.term.BNode('Nc947f08ba2154df8b104a95b228a757a'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#elementList'), rdflib.term.BNode('N2a1b234a1c4043a5b195287cec126238'))


In [70]:
qVariant = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                SELECT * WHERE  {{
               N6c29fe98927b417cb552d5e0a15999df ?p ?variant .
              
                }}"""
r = graph.query(qVariant)
r.bindings

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found 'N6c29fe98927b417'  (at char 253), (line:5, col:16)

In [45]:
def ParserAgents(graph, authority):
    # Type
    tipo = GetType(graph, authority)

    # adminMetadata
    adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]}
    
    obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'}
    
    # ElementList
    obj = getElementList(graph, authority, obj)
    
    # fullerName
    obj = getFullerName(graph, authority, obj)

    # hasCloseExternalAuthority
    obj = GetUri(authority, graph, "hasCloseExternalAuthority", obj)

    # hasExactExternalAuthority
    obj = GetUri(authority, graph, "hasExactExternalAuthority", obj)

    return obj



obj = ParserAgents(graph, authority)
obj

{'type': 'PersonalName',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'n80002329'}]},
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/PersonalName/',
 'elementList': [{'type': 'FullNameElement',
   'elementValue': {'value': 'Machado de Assis,', 'lang': None}},
  {'type': 'DateNameElement',
   'elementValue': {'value': '1839-1908', 'lang': None}}],
 'fullerName': {'type': 'PersonalName',
  'elementValue': {'value': 'Joaquim Maria Machado', 'lang': None}},
 'hasCloseExternalAuthority': [{'value': 'http://www.wikidata.org/entity/Q311145',
   'base': 'www.wikidata.org',
   'label': {'value': 'Joaquim Maria Machado de Assis', 'lang': None}},
  {'value': 'http://id.worldcat.org/fast/48407',
   'base': 'id.worldcat.org',
   'label': {'value': 'Machado de Assis, 1839-1908', 'lang': None}}]}

In [46]:
obj = GetUri(authority, graph, "hasExactExternalAuthority", {})
obj

{}

In [26]:
request = Agents(**obj)
request.hasCloseExternalAuthority

[Uri(value=rdflib.term.URIRef('http://www.wikidata.org/entity/Q311145'), label=Label(value='Joaquim Maria Machado de Assis', lang=None), base='www.wikidata.org'),
 Uri(value=rdflib.term.URIRef('http://id.worldcat.org/fast/48407'), label=Label(value='Machado de Assis, 1839-1908', lang=None), base='id.worldcat.org')]

In [35]:
x = request.hasCloseExternalAuthority[0]
value = x.value
value.toPython()

'http://www.wikidata.org/entity/Q311145'